<a href="https://colab.research.google.com/github/polpranav/Information-Retrieval-System/blob/main/Pranav_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install langchain langchain-community langchainhub chromadb pypdf groq pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:0

In [1]:
pip install --upgrade langchain


In [2]:
!pip install -U langchain-groq

In [3]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [4]:

from langchain_groq import ChatGroq  # Updated import

In [5]:
# ✅ Upload PDF files in Colab
from google.colab import files
uploaded = files.upload()

Saving FY25_Q1_Consolidated_Financial_Statements.pdf to FY25_Q1_Consolidated_Financial_Statements.pdf


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq  # ✅ Use langchain-groq package


# ✅ Load all uploaded PDFs
all_documents = []
for filename in uploaded.keys():
    loader = PyMuPDFLoader(f"/content/{filename}")
    docs = loader.load()
    all_documents.extend(docs)

# 🧩 Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(all_documents)

# 📊 Embed and store in Chroma
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="./chroma_db")
retriever = vector_store.as_retriever()

# 🧠 Use LLaMA3-70B from Groq
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)

# 🔄 Set up RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 🚀 Main loop
print("\n📄 PDF Q&A Bot (Groq + LLaMA3-70B)")
print("Ask anything from the PDF. Type 'exit' to quit.\n")

while True:
    query = input("🧠 You: ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Bye!")
        break

    result = qa_chain({"query": query})
    answer = result['result'].strip()

    # Handle empty/irrelevant answers
    if not answer or answer.lower() in ["i don't know", "i cannot answer that", ""]:
        print("\n❌ Sorry, I couldn’t find the answer in the document.\n")
    else:
        print("\n🤖 Answer:", answer)

    # Optional: show source info
    print("\n📚 Source(s):")
    for doc in result['source_documents']:
        print(f"• {doc.metadata.get('source', 'Unknown')} | Page: {doc.metadata.get('page', 'N/A')}")
    print("\n" + "-"*60 + "\n")


<ipython-input-6-ff7d2f191610>:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


📄 PDF Q&A Bot (Groq + LLaMA3-70B)
Ask anything from the PDF. Type 'exit' to quit.

🧠 You: what is sale of apple 


<ipython-input-6-ff7d2f191610>:46: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



🤖 Answer: According to the provided financial statements, the total net sales" of Apple Inc. for the three months ended December 28, 2024, is $124,300 million.

Breaking it down further, we have:

* Net sales by category:
	+ iPhone: $69,138 million
	+ Mac: $8,987 million
	+ iPad: $8,088 million
	+ Wearables, Home and Accessories: $11,747 million
	+ Services: $26,340 million

Let me know if you'd like me to clarify anything else!

📚 Source(s):
• /content/FY25_Q1_Consolidated_Financial_Statements.pdf | Page: 0
• /content/FY25_Q1_Consolidated_Financial_Statements.pdf | Page: 0
• /content/FY25_Q1_Consolidated_Financial_Statements.pdf | Page: 1
• /content/FY25_Q1_Consolidated_Financial_Statements.pdf | Page: 2

------------------------------------------------------------

🧠 You: which sector is perfoming good 

🤖 Answer: Based on the provided data, it appears that the "iPhone" category is performing well, with net sales of $69,138 and $69,702 in the two periods, respectively. This category